In [9]:
import random
import numpy as np

In [10]:
class Network(object):
    def __init__(self, sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]
        
    def feedforward(self, a):
        """
            Return the output of the network.
            a is input
        """
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a) + b)
        return a
    
    def sgd(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        if test_data:
            n_test = len(test_data)
        n = len(training_data)
        
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size] for k in range(0, n, mini_batch_size)
            ]
            
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            
            if test_data:
                print("Epoch {0}: {1} / {2}".format(j, self.evaluate(test_data), n_test))
            else:
                print("Epoch {0} complete".format(j))
                
    def update_mini_batch(self, mini_batch, eta):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        
        self.weights = [w - (eta / len(mini_batch)) * nw for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b - (eta / len(mini_batch)) * nb for b, nb in zip(self.biases, nabla_b)]
    
    def backprop(self, x, y):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        activation = x
        activations = [x]
        zs = []
        
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation) + b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        
        delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)
    
    def evaluate(self, test_data):
        test_results = [(np.argmax(self.feedforward(x)), y) for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)
    
    def cost_derivative(self, output_activations, y):
        return (output_activations - y)

In [11]:
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

def sigmoid_prime(z):
    return sigmoid(z) * (1 - sigmoid(z))

In [12]:
import mnist_loader

In [13]:
training_data, validation_data, test_data = mnist_loader.load_data_wrapper()

In [14]:
training_data, validation_data, test_data = list(training_data), list(validation_data), list(test_data)

In [15]:
net = Network([784, 30, 10])

In [16]:
net.sgd(training_data, 30, 10, 3.0, test_data=test_data)

Epoch 0: 8228 / 10000
Epoch 1: 8383 / 10000
Epoch 2: 9304 / 10000
Epoch 3: 9357 / 10000
Epoch 4: 9406 / 10000
Epoch 5: 9400 / 10000
Epoch 6: 9407 / 10000
Epoch 7: 9442 / 10000
Epoch 8: 9450 / 10000
Epoch 9: 9460 / 10000
Epoch 10: 9486 / 10000
Epoch 11: 9469 / 10000
Epoch 12: 9453 / 10000
Epoch 13: 9446 / 10000
Epoch 14: 9486 / 10000
Epoch 15: 9503 / 10000
Epoch 16: 9470 / 10000
Epoch 17: 9488 / 10000
Epoch 18: 9491 / 10000
Epoch 19: 9494 / 10000
Epoch 20: 9500 / 10000
Epoch 21: 9507 / 10000
Epoch 22: 9530 / 10000
Epoch 23: 9502 / 10000
Epoch 24: 9493 / 10000
Epoch 25: 9510 / 10000
Epoch 26: 9499 / 10000
Epoch 27: 9485 / 10000
Epoch 28: 9522 / 10000
Epoch 29: 9505 / 10000


In [17]:
net = Network([784, 100, 10])

In [18]:
net.sgd(training_data, 30, 10, 3.0, test_data=test_data)

Epoch 0: 6357 / 10000
Epoch 1: 6475 / 10000
Epoch 2: 6529 / 10000
Epoch 3: 6575 / 10000
Epoch 4: 6604 / 10000
Epoch 5: 6641 / 10000
Epoch 6: 6680 / 10000
Epoch 7: 6728 / 10000
Epoch 8: 7484 / 10000
Epoch 9: 7504 / 10000
Epoch 10: 7532 / 10000
Epoch 11: 7561 / 10000
Epoch 12: 8642 / 10000
Epoch 13: 8604 / 10000
Epoch 14: 8657 / 10000
Epoch 15: 8682 / 10000
Epoch 16: 8675 / 10000
Epoch 17: 8673 / 10000
Epoch 18: 8685 / 10000
Epoch 19: 8657 / 10000
Epoch 20: 8685 / 10000
Epoch 21: 8688 / 10000
Epoch 22: 8685 / 10000
Epoch 23: 8681 / 10000
Epoch 24: 8697 / 10000
Epoch 25: 8702 / 10000
Epoch 26: 8702 / 10000
Epoch 27: 8699 / 10000
Epoch 28: 8701 / 10000
Epoch 29: 8702 / 10000
